In [4]:
# Analyze simulation data

import pandas as pd
import ast
import json
import teams.teams_helpers as teams_helpers
import params_team as params
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import math
import os
from termcolor import colored
import warnings
import textwrap
import pickle
from ast import literal_eval
import copy
plt.rcParams['figure.figsize'] = [15, 10]

import teams.teams_helpers as team_helpers
import params_team as params


warnings.simplefilter(action='ignore', category=FutureWarning)

In [66]:
def run_sensitivity_analysis(path, files, file_prefix_list, runs_to_exclude_list=[], runs_to_analyze_list = [], vars_filename_prefix = ''):

    
    def convert_to_dict(sim_vars, parameter_combinations):
        demo_strategy = sim_vars['demo_strategy'].iloc[0]
        team_composition = sim_vars['team_composition'].iloc[0]

        study_id = int(sim_vars['study_id'].iloc[0])
        run_id = sim_vars['run_no'].iloc[0] 

        max_learning_factor = np.round(sim_vars['max_learning_factor'].iloc[0],2)
        team_learning_factor = sim_vars['initial_team_learning_factor'].iloc[0]
        # team_learning_rate = sim_vars['team_learning_rate'].iloc[0]


        learning_factor_high_learner = []
        learning_factor_low_learner = []
        for i in range(len(team_composition)):
            if team_composition[i] == 0:
                learning_factor_low_learner = np.round(team_learning_factor[i],2)
            elif team_composition[i] == 2:
                learning_factor_high_learner = np.round(team_learning_factor[i],2)

        # for i in range(len(team_learning_rate)):
        #     team_learning_rate[i] = np.round(team_learning_rate[i], 2)
        

        max_loop_count = sim_vars['loop_count'].iloc[-1]
        bec_final = sim_vars['BEC_knowledge_level'][len(sim_vars)-1]


        
        sensitivity_data_dict = {}

        sensitivity_data_dict['demo_strategy'] = demo_strategy
        sensitivity_data_dict['team_composition'] = team_composition
        sensitivity_data_dict['study_id'] = study_id
        sensitivity_data_dict['run_id'] = run_id
        
        sensitivity_data_dict['max_learning_factor'] = max_learning_factor
        sensitivity_data_dict['learning_factor_high_learner'] = learning_factor_high_learner
        sensitivity_data_dict['learning_factor_low_learner'] = learning_factor_low_learner

        sensitivity_data_dict['teacher_learning_factor'] = np.round(parameter_combinations[study_id-1][4],2)
        sensitivity_data_dict['learning_rate'] = np.round(parameter_combinations[study_id-1][2],2)
        
        sensitivity_data_dict['parameter_combinations'] = parameter_combinations[study_id-1]
        # sensitivity_data_dict['team_learning_rate'] = team_learning_rate

        sensitivity_data_dict['max_loop_count'] = int(max_loop_count)
        sensitivity_data_dict['bec_final'] = bec_final


        return sensitivity_data_dict

    #####################
    # read parameter combinations
    with open('data/simulation/sim_experiments/sensitivity_analysis/w_feedback/param_combinations_forte.pickle', 'rb') as f:
            parameter_combinations = pickle.load(f)

    
    params_pd = pd.DataFrame()
    for i in range(len(parameter_combinations)):
        params_dict = {'study_id': i+1, 'learning_factor_low': parameter_combinations[i][0], 'learning_factor_high': parameter_combinations[i][1], 'learning_rate': parameter_combinations[i][2], 'max_learning_factor': parameter_combinations[i][3], 'teacher_learning_factor': parameter_combinations[i][4]}
        params_pd = params_pd.append(params_dict, ignore_index=True)

    params_pd.to_csv('data/simulation/sim_experiments/sensitivity_analysis/w_feedback/param_combinations_forte.csv')

#     print('parameter_combinations:', parameter_combinations)



#     #############
    run_no = 1
    sensitivity_data = pd.DataFrame()
    for file in files:

        # check if file is a valid file
        for file_prefix in file_prefix_list:
            if file_prefix in file and '.pickle' in file:
                run_file_flag = True
                break
            else:
                run_file_flag = False

        # check if file needs to be excluded
        for runs_to_exclude in runs_to_exclude_list:
            if runs_to_exclude in file:
                run_file_flag = False
                break

        
        if run_file_flag:
            
            with open(path + '/' + file, 'rb') as f:
                sim_vars = pickle.load(f)
            
            print('Reading file: ', file)

            # check if there are multiple runs in the same file
            loop_count_var = sim_vars['loop_count']
            run_change_idx = [idx for idx in range(len(loop_count_var)-1) if loop_count_var[idx] > loop_count_var[idx+1]]

#             print('run_change_idx: ', run_change_idx)
            if len(run_change_idx) > 0:

                for run_idx in range(2):
                    if run_idx == 0:
                        run_sim_vars = sim_vars.iloc[:run_change_idx[0]+1]
                    else:
                        run_sim_vars = sim_vars.iloc[run_change_idx[0]+2:]
                        
                    # reset index
                    run_sim_vars = run_sim_vars.reset_index(drop=True)


                    bec_final = run_sim_vars['BEC_knowledge_level'][len(run_sim_vars)-1]
                    # check if learning was completed
                    learning_complete = True
                    for k_type, k_val in bec_final.items():
                        if k_val[0] != 1:
                            learning_complete = False
                            break
                    
                    if learning_complete:
                        sensitivity_data_dict = convert_to_dict(run_sim_vars, parameter_combinations)
                        sensitivity_data = sensitivity_data.append(sensitivity_data_dict, ignore_index=True)
            else:

                bec_final = sim_vars['BEC_knowledge_level'][len(sim_vars)-1]
                # check if learning was completed
                learning_complete = True
                for k_type, k_val in bec_final.items():
                    if k_val[0] != 1:
                        learning_complete = False
                        break
            
                if learning_complete:
                    sensitivity_data_dict = convert_to_dict(sim_vars, parameter_combinations)
                    sensitivity_data = sensitivity_data.append(sensitivity_data_dict, ignore_index=True)


    sensitivity_data.to_csv(path + '/sensitivity_data.csv')
    with open(path + '/sensitivity_data.pickle', 'wb') as f:
        pickle.dump(sensitivity_data, f)

    print(sensitivity_data)

In [58]:
# # ## Sensitivity Analysis
path = 'data/simulation/sim_experiments/sensitivity_analysis/w_feedback/augmented_taxi2'
files = os.listdir(path)

file_prefix_list = ['trials_02_11_sensitivity_w_feedback_tc1_ck']

In [65]:
run_sensitivity_analysis(path, files, file_prefix_list, runs_to_exclude_list=[], runs_to_analyze_list = [], vars_filename_prefix = '')

Reading file:  trials_02_11_sensitivity_w_feedback_tc1_ck_study_1_run_4.pickle
Reading file:  trials_02_11_sensitivity_w_feedback_tc1_ck_study_3_run_1.pickle
Reading file:  trials_02_11_sensitivity_w_feedback_tc1_ck_study_3_run_2.pickle
Reading file:  trials_02_11_sensitivity_w_feedback_tc1_ck_study_4_run_5.pickle
Reading file:  trials_02_11_sensitivity_w_feedback_tc1_ck_study_5_run_1.pickle
Reading file:  trials_02_11_sensitivity_w_feedback_tc1_ck_study_5_run_3.pickle
Reading file:  trials_02_11_sensitivity_w_feedback_tc1_ck_study_9_run_4.pickle
Reading file:  trials_02_11_sensitivity_w_feedback_tc1_ck_study_10_run_1.pickle
Reading file:  trials_02_11_sensitivity_w_feedback_tc1_ck_study_10_run_2.pickle
Reading file:  trials_02_11_sensitivity_w_feedback_tc1_ck_study_11_run_1.pickle
Reading file:  trials_02_11_sensitivity_w_feedback_tc1_ck_study_12_run_2.pickle
Reading file:  trials_02_11_sensitivity_w_feedback_tc1_ck_study_13_run_3.pickle
Reading file:  trials_02_11_sensitivity_w_feedb

Reading file:  trials_02_11_sensitivity_w_feedback_tc1_ck_study_13_run_2.pickle
Reading file:  trials_02_11_sensitivity_w_feedback_tc1_ck_study_13_run_4.pickle
Reading file:  trials_02_11_sensitivity_w_feedback_tc1_ck_study_14_run_3.pickle
Reading file:  trials_02_11_sensitivity_w_feedback_tc1_ck_study_15_run_1.pickle
Reading file:  trials_02_11_sensitivity_w_feedback_tc1_ck_study_16_run_1.pickle
Reading file:  trials_02_11_sensitivity_w_feedback_tc1_ck_study_17_run_3.pickle
Reading file:  trials_02_11_sensitivity_w_feedback_tc1_ck_study_18_run_1.pickle
Reading file:  trials_02_11_sensitivity_w_feedback_tc1_ck_study_19_run_1.pickle
Reading file:  trials_02_11_sensitivity_w_feedback_tc1_ck_study_19_run_3.pickle
Reading file:  trials_02_11_sensitivity_w_feedback_tc1_ck_study_19_run_5.pickle
Reading file:  trials_02_11_sensitivity_w_feedback_tc1_ck_study_21_run_4.pickle
Reading file:  trials_02_11_sensitivity_w_feedback_tc1_ck_study_22_run_3.pickle
Reading file:  trials_02_11_sensitivity_

In [55]:
with open (path + '/sensitivity_data.pickle', 'rb') as f:
    sensitivity_data = pickle.load(f)

In [56]:
sensitivity_data

demo_strategy team_composition  study_id  run_id  \
0             common_knowledge        [0, 0, 0]         1       4   
1    individual_knowledge_high        [0, 0, 0]         1       4   
2             common_knowledge        [0, 0, 0]         3       1   
3    individual_knowledge_high        [0, 0, 0]         3       1   
4             common_knowledge        [0, 0, 0]         3       2   
..                         ...              ...       ...     ...   
287  individual_knowledge_high        [0, 0, 0]        26       1   
288           common_knowledge        [0, 0, 0]        28       4   
289  individual_knowledge_high        [0, 0, 0]        28       4   
290           common_knowledge        [0, 0, 0]        30       3   
291  individual_knowledge_high        [0, 0, 0]        30       3   

     max_learning_factor  learning_factor_high_learner  \
0                   0.95                          0.71   
1                   0.95                          0.71   
2                   0.91                          0.70   
3                   0.91                          0.74   
4                   0.91                          0.66   
..                   ...                           ...   
287                 0.92                          0.71   
288                 0.93                          0.73   
289                 0.93                          0.77   
290                 0.87                          0.66   
291                 0.87                          0.71   

    learning_factor_low_learner  teacher_learning_factor  learning_rate  \
0                            []                 0.857353       0.055538   
1                            []                 0.857353       0.055538   
2                            []                 0.674489       0.061297   
3                            []                 0.674489       0.061297   
4                            []                 0.674489       0.061297   
..                          ...                      ...            ...   
287                          []                 0.830671       0.063368   
288                          []                 0.600125       0.070761   
289                          []                 0.600125       0.070761   
290                          []                 0.637519       0.000406   
291                          []                 0.637519       0.000406   

                                parameter_combinations  max_loop_count  \
0    [0.6718128945981977, 0.777999972109676, 0.0555...               7   
1    [0.6718128945981977, 0.777999972109676, 0.0555...               7   
2    [0.6146988332390071, 0.8455006172046735, 0.061...               6   
3    [0.6146988332390071, 0.8455006172046735, 0.061...               8   
4    [0.6146988332390071, 0.8455006172046735, 0.061...               9   
..                                                 ...             ...   
287  [0.6749770645683657, 0.8300326850743134, 0.063...               9   
288  [0.685646369817164, 0.7635257237653668, 0.0707...               9   
289  [0.685646369817164, 0.7635257237653668, 0.0707...               7   
290  [0.6770631817657633, 0.84972556577203, 0.00040...               9   
291  [0.6770631817657633, 0.84972556577203, 0.00040...               7   

                                             bec_final  
0    {'p1': [1.0], 'p2': [1.0], 'p3': [1.0], 'commo...  
1    {'p1': [1.0], 'p2': [1.0], 'p3': [1.0], 'commo...  
2    {'p1': [1.0], 'p2': [1.0], 'p3': [1.0], 'commo...  
3    {'p1': [1.0], 'p2': [1.0], 'p3': [1.0], 'commo...  
4    {'p1': [1.0], 'p2': [1.0], 'p3': [1.0], 'commo...  
..                                                 ...  
287  {'p1': [1.0], 'p2': [1.0], 'p3': [1.0], 'commo...  
288  {'p1': [1.0], 'p2': [1.0], 'p3': [1.0], 'commo...  
289  {'p1': [1.0], 'p2': [1.0], 'p3': [1.0], 'commo...  
290  {'p1': [1.0], 'p2': [1.0], 'p3': [1.0], 'commo...  
291  {'p1': [1.0], 'p2': [1.0], 'p3': [1.0], 'commo...  

[292 rows x 12 c

In [68]:
from pingouin import partial_corr

In [69]:
sensitivity_data_short = sensitivity_data[['max_loop_count', 'max_learning_factor', 'learning_factor_high_learner', 'teacher_learning_factor', 'learning_rate']]


# Compute partial rank correlation coefficients
partial_corr_results = {}
var_list = ['max_learning_factor', 'learning_factor_high_learner', 'teacher_learning_factor', 'learning_rate']
for var in var_list:
    covars = [var2 for var2 in var_list if var2 != var]

    partial_corr_results[var] = partial_corr(data=sensitivity_data_short, x='max_loop_count', y=var, covar=covars, method='pearson')

# Print the results
print("Partial Rank Correlation Coefficients:")
print(partial_corr_results)

# fig, ax = plt.subplots(1, len(var_list), figsize=(10, 6))

# for i, var in enumerate(var_list):
#     ax[i].scatter(sensitivity_data_short[var], sensitivity_data_short['max_loop_count'])
#     ax[i].set_xlabel(var)
#     ax[i].set_ylabel('N_interactions')
#     ax[i].set_title('Partial Rank Correlation Coefficient')

# plt.show()

# Calculate correlation coefficients
correlation_matrix = sensitivity_data_short.corr()

# Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap between Input Parameters and Output')
plt.show()

TypeError: cov() got an unexpected keyword argument 'numeric_only'